# Лабораторна робота 4: Advanced Nets

## 1. Передбачення тексту (seq2vec)

### Обробка та класифікація текстових даних

Візьмемо той же самий датасет із текстами (тексти постів із твіттера)

In [1]:
import pandas as pd
import numpy as np

tweets = pd.read_csv("gender_tweets.csv")
tweets = tweets[["gender", "text"]]
tweets = tweets.loc[(tweets["gender"] == "male") | (tweets["gender"] == "female")]
tweets = tweets.dropna()

print(len(tweets.index))

12894


In [2]:
# from nltk import download
# download('stopwords')
from nltk.corpus import stopwords
import re
import string


stop_words = stopwords.words("english")

def clean_text(text):
    text = text.lower()
    text = re.sub("@\S+", " ", text)
    text = re.sub("https*\S+", " ", text)
    text = re.sub("#\S+", " ", text)
    text = re.sub("\d", " ", text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', ' ', text)
    # text = re.sub('щч', ' ', text) 
    text = re.sub('\s{2,}', ' ', text)
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text


texts = tweets["text"]
tweets["text"] = [clean_text(t) for t in texts]

tweets.head(10)

,gender,text
0,male,robbie e responds critics win eddie edwards
1,male,‰ыпit felt like friends living story them‰ыќ
2,male,absolutely adore louis starts songs hits hard ...
3,male,hi looking url use typically see advanced user
4,female,watching neighbours sky catching neighbs xxx щ...
5,female,ive seen people train lamps chairs tvs etc
7,male,gala bingo clubs bought еј uk largest high str...
8,female,pic defines mcd fangirls fanboys mcd shippers xd
9,female,lovely tree year never seen gorgeous
12,female,put ass line repay


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

max_features = 10000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(tweets["text"])

tweets_words = tokenizer.texts_to_sequences(tweets["text"])
tweets_words = [b for b in tweets_words if len(b) > 3]

tweets_sent = [sent[:-1] for sent in tweets_words]
tweets_next_word = np.array([sent[-1] for sent in tweets_words], dtype='float64')

tweets_sent_padd = keras.utils.pad_sequences(tweets_sent, maxlen=5, padding='pre', truncating='pre')

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(tweets_sent_padd, tweets_next_word, test_size=0.25)

In [5]:

model = keras.Sequential([
    layers.Embedding(max_features, 128),
    layers.LSTM(128, return_sequences=True),
    layers.LSTM(128, return_sequences=False),

    layers.Dense(256, activation="relu"),
    layers.Dense(max_features, activation="softmax")
])
model.summary()


model.compile(optimizer="nadam", loss='sparse_categorical_crossentropy', metrics=["accuracy"])

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:

history = model.fit(X_train,
                    Y_train,
                    epochs=25,
                    batch_size=1500,
                    validation_data=(X_test, Y_test),
                    verbose = 1)



Epoch 1/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 265ms/step - accuracy: 0.0135 - loss: 9.2084 - val_accuracy: 0.0185 - val_loss: 9.1920
Epoch 2/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 190ms/step - accuracy: 0.0195 - loss: 9.1158 - val_accuracy: 0.0185 - val_loss: 8.5686
Epoch 3/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 189ms/step - accuracy: 0.0195 - loss: 8.0525 - val_accuracy: 0.0185 - val_loss: 8.3545
Epoch 4/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 189ms/step - accuracy: 0.0189 - loss: 7.4877 - val_accuracy: 0.0185 - val_loss: 8.3808
Epoch 5/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 190ms/step - accuracy: 0.0192 - loss: 7.3751 - val_accuracy: 0.0185 - val_loss: 8.4889
Epoch 6/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 192ms/step - accuracy: 0.0213 - loss: 7.3354 - val_accuracy: 0.0185 - val_loss: 8.6035
Epoch 7/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 189ms/step - accuracy: 0.0210 - loss: 7.3001 - val_accuracy: 0.0185 - val_loss: 8.7108
Epoch 8/25
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 190ms/step - accuracy: 0.0209 - loss: 7.2826 - val_accuracy: 0.0185 - val_loss:

Не виходить якось нормально натренувати :(

In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 5, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 5, 128)         │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10000)          │     2,570,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,438,579 (47.45 MB)

 Trainable params: 4,146,192 (15.82 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8,292,387 (31.63 MB)

In [16]:
print("контекст для передбачення: " +"\"seen people train lamps\"")
textt = tokenizer.texts_to_sequences(["seen people train lamps"])
print(textt)

index = np.argmax(model.predict(np.array(textt)))
print(index)
print("Передбачене слово: " + tokenizer.sequences_to_texts([[index]])[0])

контекст для передбачення: "seen people train lamps"
[[222, 10, 693, 8065]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
30
Передбачене слово: lol


Чи я очікував чогось іншого від текстів з твіттера? Ні.